In [1]:
from IPython.core.display import display, HTML
import sys
display(HTML("<style>.container { width:95% !important; }</style>"))
print(sys.version)

3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]


In [2]:
from datetime import date, datetime
import numpy as np
import pandas as pd
import dataframe_image as dfi
# pd.options.display.float_format = '{:,.3f}'.format
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
sns.set_theme()
sns.set_color_codes()
#pd.set_option('display.max_rows', 100)
#pd.set_option('display.max_columns', 20)

# column_names = ['# Seen', 'ALSA', '# Picked', 'ATA', '# GP', 'GP WR', '# OH', 'OH WR', '# GD', 'GD WR', '# GIH', 'GIH WR', '# GND', 'GND WR', 'IWD', 'Color', 'Rarity']

from Utilities import Logger
import WUBRG
from WUBRG import get_color_identity
from game_metadata import SETS, FORMATS
from game_metadata import Card,CardManager, SetMetadata, FormatMetadata
from data_fetching import DataLoader, LoadedData, DataFramer, FramedData, SetManager, CentralManager
from data_fetching.utils import get_next_17lands_update_time, get_prev_17lands_update_time, get_name_slice, get_color_slice, get_date_slice


LOAD_ALL = False
TRGT_SET = 'SNC'
LOG_LEVEL = Logger.FLG.DEFAULT
LOG_LEVEL

<Flags.DEFAULT: 3>

# Objects

In [3]:
import sys
try:
    del sys.modules["data_graphing"] 
    del data_graphing
except:
    pass

from data_graphing import ColorHandler, PlotterHelper, prettify_frame

In [4]:
class FramedDataFuncs:   
    def __init__(self, DATA):
        self._DATA = DATA
        self._ARCH_FUNCS = ArchFuncs(self._DATA)
        self._SNGL_CARD_FUNCS = SingleCardFuncs(self._DATA)
    
    @property
    def SET(self):
        """The draft set."""
        return self._DATA.SET
    
    @property
    def FULL_SET(self):
        """The full name of the draft set."""
        return self._DATA.FULL_SET
    
    @property
    def FORMAT(self):
        """The format type."""
        return self._DATA.FORMAT
    
    @property
    def SHORT_FORMAT(self):
        """The shorthand of the format type."""
        return self._DATA.FULL_FORMAT
    
    @property
    def DATA(self):
        """The object which contains the data about the set and format."""
        return self._DATA
        
    def check_for_updates(self):
        """Populates and updates all data properties, filling in missing data."""
        self._DATA.check_for_updates()

    def reload_data(self):
        """Populates and updates all data properties, reloading all data."""
        self._DATA.reload_data()  
    
    def deck_group_frame(self, name=None, date=None, summary=False):
        """Returns a subset of the 'GROUPED_ARCHETYPE' data as a DataFrame."""
        return self.DATA.deck_group_frame(name, date, summary)
     
    def deck_archetype_frame(self, deck_color=None, date=None, summary=False):
        """Returns a subset of the 'SINGLE_ARCHETYPE' data as a DataFrame."""
        return self.DATA.deck_archetype_frame(deck_color, date, summary)
    
    def card_frame(self, name=None, deck_color=None, date=None, card_color=None, card_rarity=None, summary=False):
        """Returns a subset of the 'CARD' data as a DataFrame."""
        return self.DATA.card_frame(name, deck_color, date, card_color, card_rarity, summary)
    
    def compress_date_range_data(self, start_date, end_date, card_name=None):
        """Summarizes card data over a provided set of time."""
        return self.DATA.compress_date_range_data(start_date, end_date, card_name)
        
    
    #ArchFuncs
    def get_games_played(self, deck_color):
        return self._ARCH_FUNCS.get_games_played(deck_color)
        
    def get_avg_winrate(self, day=None, arch='All Decks'):
        return self._ARCH_FUNCS.get_avg_winrate(day, arch)
    
    def get_archetype_frame(self, colors, roll=None):
        return self._ARCH_FUNCS.get_archetype_frame(colors, roll)

    def get_archetype_winrate_history(self, color_filter=None, roll=None, *, save=False):
        return self._ARCH_FUNCS.get_archetype_winrate_history(color_filter, roll, save=save)

    def plot_archetype_winrate_history(self, color_filter=None, roll=None, color_dict=None, pref=''):
        return self._ARCH_FUNCS.plot_archetype_winrate_history(color_filter, roll, color_dict, pref)       
        
    def get_archetype_playrate_history(self, color_filter=None,  color_count=0, roll=None, *, save=False):
        return self._ARCH_FUNCS.get_archetype_playrate_history(color_filter, color_count, roll, save=save)

    def plot_archetype_playrate_history(self, color_filter=None, color_count=0, roll=None, color_dict=None, pref=''):
        return self._ARCH_FUNCS.plot_archetype_playrate_history(color_filter, color_count, roll, color_dict, pref)
    
    def card_relative_winrates(self, deck_colors, win_rate_col, filter_option=None):
        return self._ARCH_FUNCS.card_relative_winrates(deck_colors, win_rate_col, filter_option)
    
    #SingleCardFuncs
    def get_card_summary(self, card_name, colors='', roll=None):
        return self._SNGL_CARD_FUNCS.get_card_summary(card_name, colors, roll)

    def plot_card_summary(self, card_name, colors='', roll=None):
        return self._SNGL_CARD_FUNCS.plot_card_summary(card_name, colors, roll)
    
    def get_pick_stats(self, card_name, roll=None):
        return self._SNGL_CARD_FUNCS.get_pick_stats(card_name, roll)
    
    def plot_pick_stats(self, card_name, roll=None):
        return self._SNGL_CARD_FUNCS.plot_pick_stats(card_name, roll)
    
    def card_archetype_performance(self, card_name):
        return self._SNGL_CARD_FUNCS.card_archetype_performance(card_name)
    
    def stat_archetype_performance(self, stat_name, color_cols=None, min_colors=0, max_colors=5):
        return self._SNGL_CARD_FUNCS.stat_archetype_performance(stat_name, color_cols, min_colors, max_colors)

##    # filters = ['exact', 'subset', 'contains', 'adjascent'*]
##    ## TODO: Determine how to handle colourless cards.
##    ## 'exact': 'U' --> 'U'
##    ## 'subset': 'UW' --> 'U', 'W', 'WU'
##    ## 'contains': 'U' --> 'U', 'UW', 'UB', 'UR', 'UG'...
##    ## 'contains': 'UW' --> 'UW', 'UBW', 'URW', 'UGW'...
##    ## 'superset': 'UW' --> 'UW', 'UBW', 'URW', 'UGW'...
##    ## 'adjascent': 'UW' --> 'U', 'W', 'UW', 'UG', 'WG', 'UWG'...
##    def card_color_filter(frame, card_color=None, filter_style=''):
##        if card_color is None: 
##            return frame
##        
##            card_color = WUBRG.get_color_identity(card_color)
##            frame = frame[frame['Color'] == card_color]
##            return frame

    
    def compare_card_evaluations(self, start_date, end_date):
        def inner_func(date):
            df = self.card_frame(date=date, deck_color='')
            df.index = [tup[2] for tup in df.index]
            return df

        first = inner_func(date=start_date)
        last = inner_func(date=end_date)
        diff = last[['ALSA', 'ATA', 'Color', 'Rarity']].copy()
        diff['ALSA Change'] = first['ALSA'] - last['ALSA']
        diff['ATA Change'] = first['ATA'] - last['ATA']
        return diff[['ALSA', 'ALSA Change', 'ATA', 'ATA Change', 'Color', 'Rarity']]


    def get_top(self, column, count=10, asc=True, card_color=None, card_rarity=None, deck_color='', play_lim=None):
        frame = self.card_frame(deck_color=deck_color, summary=True, card_rarity=card_rarity)
        frame = frame.sort_values(column, ascending=asc)

        if card_color is not None:
            card_color = WUBRG.get_color_identity(card_color)
            frame = frame[frame['Color'] == card_color]

        if play_lim is not None:
            #TODO: Fix this. 
            if type(play_lim) is float: play_lim *= self.get_games_played(deck_color)
            print(f'Minimum Games played to be included: {play_lim}')
            frame = frame[frame['# GP'] >= play_lim]

        return frame.head(count)

In [5]:
from data_graphing import ROLL, ARCHETYPES_COLOR_DICT
from WUBRG.consts import COLOR_PAIRS, COLOR_COUNT_REVERSE_MAP, COLOR_COUNT_SHORTHAND, COLOR_COUNT_SHORTHAND_MAP
from WUBRG import COLOUR_GROUPINGS

class ArchFuncs:   
    def __init__(self, DATA):
        self._DATA = DATA
        
    @staticmethod
    def _get_play_stat_frame(frame, col, roll=None, aggfunc=None):
        if roll is None: roll = ROLL
        if aggfunc is None: aggfunc = np.sum
        ret = frame[[col]]
        ret.reset_index(inplace=True, level=1)
        ret = ret.pivot_table(values=col, index=ret.index, columns='Name', dropna=False, aggfunc=aggfunc)
        ret = ret.rolling(window=roll, min_periods=1, center=True).mean()
        ret.columns.names = [col]
        return ret

    def _get_all_stat(self, stat, roll=None, aggfunc=None):
        archetypes = self._get_play_stat_frame(self._DATA.deck_archetype_frame(), stat, roll, aggfunc)
        archetypes = archetypes[[color for color in COLOR_COMBINATIONS if color in archetypes.columns]]

        groups = self._get_play_stat_frame(self._DATA.deck_group_frame(), stat, roll, aggfunc)
        groups.rename(columns=COLOR_COUNT_SHORTHAND_MAP, inplace=True)
        groups = groups[COLOR_COUNT_SHORTHAND]

        return pd.concat([groups, archetypes], axis=1)


    def get_winrates(self, roll=None):
        wins = self._get_all_stat('Wins', roll, np.sum)
        games = self._get_all_stat('Games', roll, np.sum)
        frame = (wins / games) * 100
        frame.columns.names = ['Avg. Win %']
        return frame


    def get_playrates(self, num_colors=0, roll=None):
        games = self._get_all_stat('Games', roll, np.sum)
        
        if num_colors == 0:
            frame = games.div(games['ALL'], axis='rows') * 100
            frame.columns.names = ['% of Decks']
            return frame

        true_name = COLOR_COUNT_REVERSE_MAP[num_colors].title()
        summ_id = COLOR_COUNT_SHORTHAND_MAP[true_name]

        frame = games.div(games[summ_id], axis='rows') * 100
        frame.columns.names = [f'% of {num_colors}C Decks']
        return frame
   
    def get_archetype_winrate_history(self, color_filter=None, roll=None, *, save=False):
        if roll is None: roll = ROLL
            
        frame = self.get_winrates(roll)
       
        if isinstance(color_filter, str):
            # TODO: Make this more flexible.
            #col_filt = ['ALL', f'{len(color_filter)}C'] + [col for col in COLOR_PAIRS if color_filter in col]  
            col_filt = ['ALL', '2C'] + [col for col in COLOR_PAIRS if color_filter in col]  
            frame = frame[col_filt]
        elif isinstance(color_filter, list):
            frame = frame[color_filter]
            
        if save:
            PlotterHelper(self._DATA).frame_to_png(frame, "archetype_winrate_table.png")
        
        return frame        
        
    #TODO: Implement a more generic version of this that takes in a list of deck colours to include as output. 
    def get_archetype_playrate_history(self, color_filter=None, color_count=0, roll=None, *, save=False):
        if roll is None: roll = ROLL
            
        frame = self.get_playrates(color_count, roll)

        if isinstance(color_filter, str):
            col_filt = [col for col in COLOR_PAIRS if color_filter in col]
            frame = frame[col_filt]
        if isinstance(color_filter, list):
            frame = frame[color_filter]
            
        if save:
            PlotterHelper(self._DATA).frame_to_png(frame, "archetype_playrate_table.png")

        return frame
    
    def plot_archetype_winrate_history(self, color_filter=None, roll=None, color_dict=None, pref=''):
        if roll is None: roll = ROLL
        if color_dict is None: color_dict = ARCHETYPES_COLOR_DICT

        data = self.get_archetype_winrate_history(color_filter, roll)
        data.index = [idx[5:] for idx in data.index]
        colors = str(color_filter)  #TODO: Make this cleverer at showing filter.

        plot_help = PlotterHelper(self._DATA, color_dict=color_dict)
        fig, ax = plot_help.new_single_plot('Archetype Winrates', width=16, height=8)
        plot_help.accredit(y=0.035, x=0.51)
        plot_help.desc_note(colors=colors, roll=roll, y=0.935, x=0.51)
        
        plot_help.set_labels(x_label="Date", y_label="Win Rate")
        plot_help.set_data(data, color_filter)
        
        if not pref:
            plot_help.save_fig(f"win_rates_{roll}day_avg.png", "Metagame")
        else:
            plot_help.save_fig(f"{pref}_win_rates_{roll}day_avg.png", "Metagame")

    def plot_archetype_playrate_history(self, color_filter=None, color_count=0, roll=None, color_dict=None, pref=''):
        if roll is None: roll = ROLL
        if color_dict is None: color_dict = ARCHETYPES_COLOR_DICT
            
        data = self.get_archetype_playrate_history(color_filter, color_count, roll)
        data.index = [idx[5:] for idx in data.index]
        
        
        plot_help = PlotterHelper(self._DATA, color_dict=color_dict)
        fig, ax = plot_help.new_single_plot('Archetype Playrates', width=16, height=8)
        plot_help.accredit(y=0.035, x=0.51)
        plot_help.desc_note(colors=color_filter, roll=roll, y=0.935, x=0.51)
        
        plot_help.set_labels(x_label="Date", y_label="Percent of Metagame")
        plot_help.set_data(data, color_filter)
        
        if not pref:
            plot_help.save_fig(f"play_rates_{roll}day_avg.png", "Metagame")
        else:
            plot_help.save_fig(f"{pref}_play_rates_{roll}day_avg.png", "Metagame")
            
    
    def card_relative_winrates(self, deck_colors=None, win_rate_col='GIH WR', filter_option=None):
        deck_colors = get_color_identity(deck_colors)
        
        # Get the relevant list of cards, and then trim it down to the relevant colours.
        sub_frame = self._DATA.card_frame(deck_color=deck_colors)
        sub_frame = set_data.BO1.DATA.CARD_HISTORY_FRAME.loc[slice(None), get_color_slice(deck_colors), get_name_slice(None)]
        sub_frame = sub_frame.reset_index(level=1)
        if deck_colors:
            sub_frame = sub_frame[sub_frame['Cast Color'].isin(WUBRG.get_color_subsets(deck_colors))]

        # Get the winrates for the cards and average winrate of the archetype, then re-center the card winrates with it. 
        win_frame = self._get_play_stat_frame(sub_frame, win_rate_col, roll=1, aggfunc=np.mean)
        avg_frame = self.get_archetype_winrate_history(deck_colors)
        target_avg = deck_colors if deck_colors else 'ALL'
        ret_frame = win_frame.sub(avg_frame[target_avg], axis='rows').T

        # Get the games played from the subset of cards and use it to re-weight the win rates per day, to calculate an accurate average.
        games_frame = self._get_play_stat_frame(sub_frame, '# GP', roll=1, aggfunc=np.mean).T
        ret_frame[f'AVG'] = (ret_frame * games_frame).sum(axis=1) / games_frame.sum(axis=1)
        ret_frame['# GP'] = games_frame.sum(axis=1)

        # Sort by most winning first.
        ret_frame = ret_frame.sort_values(f'AVG', ascending=False).T
        return ret_frame[::-1]

In [6]:
from data_graphing import ROLL, STATS_COLOR_DICT
from WUBRG import COLOR_COMBINATIONS
from WUBRG.consts import COLOR_PAIRS


class SingleCardFuncs:   
    def __init__(self, DATA):
        self._DATA = DATA
        self._COLOR_IDX = 0
        
    def _shorten_data(self, card_name, roll, cols, colors=''):
        frame = self._DATA.card_frame(name=card_name, deck_color=colors)[cols]
        frame.index = [tup[0][5:] for tup in frame.index]
        rolling = frame.rolling(window=roll, min_periods=1, center=True).mean()
        return rolling

    def plot_card_summary(self, card_name, colors='', roll=None, color_dict=None):
        if roll is None: roll = ROLL
        if color_dict is None: color_dict = STATS_COLOR_DICT
        rolling = self._shorten_data(card_name, roll, ['GIH WR', 'GND WR', 'ATA', 'ALSA', '# GP', '# GIH', '# Picked', '# Seen'], colors=colors)
        if rolling.empty:
            print(f"""Could not find data for "{card_name}". Please make sure it is spelled correctly, or you are accessing the right set.""")
            return
        
        plot_help = PlotterHelper(self._DATA, color_dict=color_dict)
        fig, ax = plot_help.new_quad_plot(card_name)
        plot_help.accredit(y=0.075)
        plot_help.desc_note(colors=colors, roll=roll)
                
        plot_help.set_labels(y_label="Win Percent", g_x=0, g_y=0)
        plot_help.set_data(rolling, ['GIH WR', 'GND WR'], g_x=0, g_y=0)
        
        plot_help.set_labels(y_label="Pick Number", g_x=0, g_y=1)
        plot_help.set_data(rolling, ['ALSA', 'ATA'], inv_y=True, g_x=0, g_y=1)
        
        plot_help.set_labels(x_label="Date", y_label="# of Games", g_x=1, g_y=0)
        plot_help.set_data(rolling, ['# GP', '# GIH'], g_x=1, g_y=0)

        plot_help.set_labels(x_label="Date", y_label="# of Cards", g_x=1, g_y=1)
        plot_help.set_data(rolling, ['# Seen', '# Picked'], g_x=1, g_y=1)
        
        if colors:
            plot_help.save_fig(f"pcs_{card_name}_{colors}.png", "Summary")
        else:
            plot_help.save_fig(f"pcs_{card_name}.png", "Summary")
    
    
    def plot_pick_stats(self, card_name, roll=None, color_dict=None):
        if roll is None: roll = ROLL        
        if color_dict is None: color_dict = STATS_COLOR_DICT
        taken_data = self._shorten_data(card_name, roll, ['ALSA', 'ATA'])
        if taken_data.empty:
            print(f"""Could not find data for "{card_name}". Please make sure it is spelled correctly, or you are accessing the right set.""")
            return
        
        plot_help = PlotterHelper(self._DATA, color_dict=color_dict)
        fig, ax = plot_help.new_single_plot(card_name)
        plot_help.accredit()
        plot_help.desc_note(roll=roll, y=0.96)
        
        plot_help.set_labels(x_label="Date", y_label="Pick Number")
        plot_help.set_data(taken_data, ['ALSA', 'ATA'], inv_y=True)
        
        plot_help.save_fig(f"pps_{card_name}.png", "Pick Stats")


    def card_archetype_performance(self, card_name):
        d = dict()
        d['AVG'] = self._DATA.card_frame(card_name, '', summary=True)
        for col in COLOR_PAIRS:
            d[col] = self._DATA.card_frame(card_name, col, summary=True)

        test_frame = pd.DataFrame.from_dict(d).T
        return test_frame
    
    
    def stat_archetype_performance(self, stat_name, color_cols=None, min_colors=0, max_colors=5):
        series = self._DATA.card_frame(summary=True)[stat_name]
        frame = series.reset_index(level=0)
        ret = pd.pivot_table(frame, index='Name', columns='Deck Colors')
        ret.columns = ret.columns.droplevel(0)
        #ret = ret[COLOR_COMBINATIONS]  #Re-orders the columns in WUBRG order
        if color_cols is not None:
            ret = ret[color_cols]
        return ret

# Initialization

In [7]:
data_manager = None
set_data = None

In [8]:
print(f"Current Local Time:  {datetime.now()}")
print(f"Last 17Lands Update: {get_prev_17lands_update_time()}")
print(f"Current UTC Time:    {datetime.utcnow()}")
print(f"Next 17Lands Update: {get_next_17lands_update_time()}")

Current Local Time:  2022-05-10 21:05:15.228419
Last 17Lands Update: 2022-05-10 02:00:00
Current UTC Time:    2022-05-10 23:35:15.228419
Next 17Lands Update: 2022-05-11 02:00:00


In [9]:
start = datetime.utcnow()

if LOAD_ALL:
    if data_manager is None:
        data_manager = CentralManager()
        SNC = data_manager['SNC']
        NEO = data_manager['NEO']
        VOW = data_manager['VOW']
        MID = data_manager['MID']
        set_data = data_manager[TRGT_SET]
            
    data_manager.check_for_updates()
else:
    if set_data is None:
        set_data = SetManager(TRGT_SET)
    set_data.check_for_updates()

examiner = FramedDataFuncs(set_data.BO1)

end = datetime.utcnow()
print(f"\n --- Data loaded in {end - start}.")

Loading set metadata for: SNC
Fetching data for set: SNC
Fetching card data for set: SNC
Done!

Checking for missing data for SNC PremierDraft...
Getting overall data for SNC PremierDraft
Getting data for SNC PremierDraft, date: 2022-04-28
Getting data for SNC PremierDraft, date: 2022-04-29
Getting data for SNC PremierDraft, date: 2022-04-30
Getting data for SNC PremierDraft, date: 2022-05-01
Getting data for SNC PremierDraft, date: 2022-05-02
Getting data for SNC PremierDraft, date: 2022-05-03
Getting data for SNC PremierDraft, date: 2022-05-04
Getting data for SNC PremierDraft, date: 2022-05-05
Getting data for SNC PremierDraft, date: 2022-05-06
Getting data for SNC PremierDraft, date: 2022-05-07
Getting data for SNC PremierDraft, date: 2022-05-08
Getting data for SNC PremierDraft, date: 2022-05-09
Finished checking for missing data for SNC PremierDraft.

Checking for missing data for SNC TradDraft...
Getting overall data for SNC TradDraft
Getting data for SNC TradDraft, date: 2022-0

In [10]:
examiner = FramedDataFuncs(set_data.BO1)
rel_wins_frame = examiner.card_relative_winrates('WR', 'GIH WR')
prettify_frame(rel_wins_frame)

GIH WR,Elspeth Resplendent,Ziatora's Proving Ground,Patch Up,Scuttling Butler,Halo Scarab,Brokers Initiate,Sanctuary Warden,Swooping Protector,Jetmir's Garden,Unlucky Witness,Professional Face-Breaker,Ominous Parcel,Tramway Station,Mysterious Limousine,Paragon of Modernity,Revelation of Power,Brass Knuckles,Rabble Rousing,Mage's Attendant,Knockout Blow,Inspiring Overseer,Arc Spitter,Maestros Theater,Kill Shot,Strangle,Gathering Throng,Hold for Ransom,Pyre-Sledge Arsonist,Sizzling Soloist,Wrecking Crew,"Jaxis, the Troublemaker",Exhibition Magician,Ready to Rumble,Call In a Professional,Gilded Pinions,Pugnacious Pugilist,Citizen's Crowbar,Cabaretti Courtyard,Angelic Observer,Sky Crier,Witty Roastmaster,Plasma Jockey,Raffine's Informant,Speakeasy Server,Mayhem Patrol,Buy Your Silence,Sticky Fingers,Boon of Safety,Racers' Ring,Botanical Plaza,Riveteers Requisitioner,Rumor Gatherer,Devilish Valet,Celebrity Fencer,Extraction Specialist,Antagonize,Backup Agent,Quick-Draw Dagger,Involuntary Employment,Unlicensed Hearse,Depopulate,Dapper Shieldmate,Light 'Em Up,Chrome Cat,Illuminator Virtuoso,Riveteers Initiate,Torch Breath,Hoard Hauler,Big Score,Obscura Storefront,Goldhound,"Urabrask, Heretic Praetor",Rob the Archives,Ballroom Brawlers,Refuse to Yield,Daring Escape,Halo Fountain,Raffine's Guidance,Skybridge Towers,Jackhammer,Cement Shoes,Brokers Hideout,Riveteers Overlook,Glittering Stockpile,Suspicious Bookcase,"Giada, Font of Hope",Getaway Car,Spara's Headquarters,Raffine's Tower,Xander's Lounge,Arcane Bombardment,"Luxior, Giada's Gift",Structural Assault,Waterfront District,Widespread Thieving
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
# GP,33.00,14.00,49.00,16.00,177.00,22.00,55.00,250.00,34.00,200.00,99.00,46.00,59.00,73.00,80.00,315.00,12.00,144.00,281.00,202.00,802.00,23.00,33.00,195.00,728.00,549.00,519.00,90.00,405.00,313.00,133.00,795.00,231.00,373.00,86.00,305.00,318.00,415.00,72.00,518.00,916.00,924.00,768.00,402.00,1077.00,209.00,559.00,296.00,184.00,160.00,334.00,251.00,53.00,271.00,109.00,554.00,678.00,351.00,102.00,57.00,18.00,400.00,523.00,128.00,439.00,222.00,132.00,21.00,203.00,28.00,528.00,88.00,63.00,179.00,72.00,123.00,41.00,232.00,20.00,75.00,29.00,123.00,88.00,10.00,4.00,27.00,14.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00
AVG,19.31,17.02,13.09,12.13,12.04,10.54,9.38,8.82,8.41,7.93,7.63,7.47,7.43,7.40,7.08,6.94,6.78,6.02,5.30,5.22,4.64,4.43,4.16,3.83,2.57,2.47,2.00,1.98,1.86,1.85,1.78,1.74,1.70,1.56,1.31,1.21,0.93,-0.58,-0.69,-1.09,-1.18,-1.51,-1.57,-1.63,-2.24,-2.37,-2.48,-2.75,-2.82,-3.02,-3.28,-3.74,-4.02,-4.05,-4.11,-4.25,-4.74,-4.87,-4.90,-5.07,-5.28,-5.42,-5.81,-5.94,-6.01,-6.41,-6.86,-7.10,-7.47,-7.55,-7.59,-7.65,-8.08,-8.53,-9.55,-10.09,-10.94,-11.50,-11.67,-12.29,-16.89,-17.96,-21.76,-22.69,-23.37,-24.04,-27.07,-54.17,-57.93,-57.93,nan,nan,nan,nan,nan
2022-05-09,25.82,50.82,nan,nan,50.82,0.82,nan,34.15,nan,14.46,50.82,nan,-49.18,nan,50.82,10.82,0.82,nan,50.82,-49.18,-11.68,nan,nan,-9.18,10.82,-49.18,-7.51,nan,22.25,-2.51,-32.51,-6.32,-49.18,-17.93,-15.85,0.82,0.82,-1.81,50.82,-15.85,-8.44,-9.89,-3.03,-19.18,-15.85,0.82,-24.18,10.82,0.82,nan,-9.18,0.82,nan,nan,25.82,0.82,-6.32,-15.85,-49.18,nan,nan,-24.18,-12.82,-49.18,-9.18,-18.41,-27.75,nan,-49.18,nan,0.82,-49.18,nan,-12.82,0.82,-24.18,-49.18,0.82,nan,-15.85,0.82,-9.18,-24.18,nan,nan,-49.18,-49.18,nan,nan,nan,nan,nan,nan,nan,nan
2022-05-08,-51.17,nan,nan,nan,23.83,nan,-1.17,-1.17,nan,-26.17,-51.17,23.83,nan,-1.17,48.83,6.72,nan,15.49,20.26,15.49,-3.55,nan,nan,-26.17,28.83,22.16,1.00,-17.84,-32.99,23.83,-51.17,-14.81,nan,-5.72,48.83,-5.72,32.16,15.49,nan,8.83,3.37,-1.17,-12.04,-1.17,8.83,48.83,-5.34,-13.67,-17.84,48.83,-17.84,18.83,-51.17,-1.17,-1.17,-1.17,4.00,11.33,23.83,nan,nan,18.83,-12.28,-8.32,1.00,-5.02,nan,nan,-1.17,nan,-9.07,5.97,nan,-17.84,20.26,-51.17,nan,-13.67,nan,nan,-51.17,-51.17,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2022-05-07,-2.38,nan,nan,nan,-2.38,nan,nan,-19.05,14

In [11]:
raise Exception('Stopping Auto-Run!')

Exception: Stopping Auto-Run!

# Examine Data

In [ ]:
examiner.plot_card_summary('Girder Goons')

In [ ]:
examiner.plot_pick_stats('Girder Goons')

In [ ]:
examiner.card_archetype_performance('Girder Goons')

In [ ]:
examiner.stat_archetype_performance('GIH WR', color_cols=['WU', 'WG', 'UB', 'BR', 'RG'])

In [ ]:
examiner = FramedDataFuncs(set_data.BO1)
rel_wins_frame = examiner.card_relative_winrates('WR', 'GIH WR')
rel_wins_fra

In [ ]:
games = rel_wins_frame.loc['# GP']
def format_rel_wins(v):
    if v == '---':
        return '---'
    elif v > 0:
        return f"+{round(v, 2)}%"
    else:
        return f"-{round(v, 2)}%"
rel_wins_frame = examiner.card_relative_winrates('WR', 'GIH WR')
rel_wins_frame.fillna('---', inplace=True)
prettify_frame(rel_wins_frame.style.format(format_rel_wins))

In [ ]:
examiner._DATA.card_frame(deck_color='')

## Current Tests

TODO: Update code so that
 1) Cards without data aren't treated as having a 0% win rate.
 2) Ensure that NaNs are respected so the average improvement is calculated properly.

In [ ]:
sub_frame = set_data.BO1.DATA.CARD_HISTORY_FRAME.loc[slice('2022-04-28', '2022-05-01'), get_color_slice('WR'), get_name_slice(None)]
sub_frame.reset_index(level=1, inplace=True)
sub_frame = sub_frame[sub_frame['Cast Color'].isin(WUBRG.get_color_subsets('WR'))]
sub_frame

In [ ]:
sub_frame.loc[(slice(None), slice('Depopulate', 'Depopulate', None)), :]

In [ ]:
temp_frame = sub_frame.copy()
temp_frame.reset_index(inplace=True, level=1)
#temp_frame

In [ ]:
temp_frame.pivot_table(values='GIH WR', index=temp_frame.index, columns='Name', dropna=False)

In [ ]:
# Get the winrates and average winrate of the colour group, then re-center the card winrates with it. 
win_frame = ArchFuncs._get_play_stat_frame(sub_frame, 'GIH WR', roll=1)
win_frame

In [ ]:
TRG_DECK_COLOR = 'WR'
TRG_WIN_RATE = 'GIH WR'

def card_relative_winrates(deck_colors, win_rate_col, filter_option=None):
    # Get the relevant list of cards, and then trim it down to the relevant colours.
    sub_frame = set_data.BO1.DATA.CARD_HISTORY_FRAME.loc[slice(None), get_color_slice(deck_colors), get_name_slice(None)]
    sub_frame = sub_frame.reset_index(level=1)
    sub_frame = sub_frame[sub_frame['Cast Color'].isin(WUBRG.get_color_subsets(deck_colors))]

    # Get the winrates and average winrate of the colour group, then re-center the card winrates with it. 
    win_frame = ArchFuncs._get_play_stat_frame(sub_frame, win_rate_col, roll=1, aggfunc=np.mean)
    avg_frame = examiner.get_archetype_winrate_history([deck_colors])
    centred_frame = win_frame.sub(avg_frame[deck_colors], axis='rows').T

    # Get the games played from the subset of cards and use it to re-weight the win rates per day, to calculate an accurate average.
    games_frame = ArchFuncs._get_play_stat_frame(sub_frame, '# GP', roll=1, aggfunc=np.mean).T
    ret_frame = centred_frame
    ret_frame['AVG'] = (ret_frame * games_frame).sum(axis=1) / games_frame.sum(axis=1)
    ret_frame['GAMES'] = games_frame.sum(axis=1)

    # Sort by most winning first.
    ret_frame = ret_frame.sort_values('AVG', ascending=False).T
    return sub_frame, win_frame, games_frame, avg_frame, ret_frame, centred_frame
    
frames = card_relative_winrates(TRG_DECK_COLOR, TRG_WIN_RATE)
frames[4]

In [ ]:
def gt(x, y):  #Used for function pointer shenanigans
    return x > y

def lt(x, y):  #Used for function pointer shenanigans
    return x < y

def filter_quadrant_cards_df(func1, func2, iwd_thresh=0, play_lim=0.01, card_rarity=None, deck_color=''):
    frame = set_data.BO1.card_frame(deck_color=deck_color, summary=True, card_rarity=card_rarity)
    
    # TODO: Make the mean different based on card rarity so cards aren't moved into incorrect categories.
    
    if play_lim is not None:
        if type(play_lim) is float: play_lim *= set_data.BO1.get_games_played(deck_color)
        print(f'Minimum Games played to be included: {play_lim}')
        frame = frame[frame['# GP'] >= play_lim]
        
    games_played_mean = frame['# GP'].mean()
    cards = frame[func1(frame['IWD'], iwd_thresh)]
    cards = cards[func2(cards['# GP'], games_played_mean)]
    cards = cards.sort_values('IWD', ascending=func1==lt)
    return cards


def get_trap_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(lt, gt, iwd_thresh, play_lim, card_rarity, deck_color)

def get_niche_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(gt, lt, iwd_thresh, play_lim, card_rarity, deck_color)

def get_staple_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(gt, gt, iwd_thresh, play_lim, card_rarity, deck_color)

def get_dreg_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(lt, lt, iwd_thresh, play_lim, card_rarity, deck_color)

In [ ]:
get_niche_cards(deck_color='GW', card_rarity='CU')

In [ ]:
import functools

@functools.singledispatch
def _stringify_for_date_slice(arg):
    # By default, if we don't have a special way to handle a parameter, don't change it.
    return arg

@_stringify_for_date_slice.register(date)
def _stringify_for_date_slice_date(arg):
    # Convert the date into a string, then handle that.
    return _stringify_for_date_slice(str(arg))  


@_stringify_for_date_slice.register(datetime)
def _stringify_for_date_slice_datetime(arg):
    # Isolate the date from the datetime, then handle that.
    return _stringify_for_date_slice_date(arg.date())


@functools.singledispatch
def get_date_slice(arg):
    # By default, if we don't have a special way to handle a parameter, don't change it.
    return arg

@get_date_slice.register(type(None))
def _get_date_slice_none(val):
    # If the parameter is a None, marshall it into a slice.
    return slice(val)

@get_date_slice.register(str)
@get_date_slice.register(date)
@get_date_slice.register(datetime)
def _get_date_slice_string(val):
    # If the parameter is a string, date or datetime, stringify it and marshall it into a single element list to filter properly.
    return [_stringify_for_date_slice(val)]

# TODO: Handle how to parse iterable types.
@get_date_slice.register(list)
@get_date_slice.register(dict)
@get_date_slice.register(set)
def _get_date_slice_iterable(val):
    # If the parameter is iterable, fsttempt to convert all of its elements, and return them as a list.
    return [_stringify_for_date_slice(str_date) for str_date in val]


@get_date_slice.register(tuple)
def _get_date_slice_range(val):
    # If the parameter is iterable, check if it should be a range.
    if len(val) == 2:
        # If it should be a range, get the colours check that the colours elements are properly ordered.
        str_date1 = _stringify_for_date_slice(val[0])
        str_date2 = _stringify_for_date_slice(val[1])
        return slice(str_date1, str_date2)
        
        # TODO: Check date ordering
        # Then make sure a valid range is returned.
        if color_compare(col2, col1) == 1:
            return slice(col1, col2)
        else:
            # TODO: See if returning a reversed range is possible.
            return slice(col2, col1)
    else:
        # If not, handle it like as an iterable.
        return _get_date_slice_iterable(val)

In [ ]:
get_date_slice(None)

In [ ]:
get_date_slice('2022-05-09')

In [ ]:
get_date_slice(date(2022, 5, 9))

In [ ]:
get_date_slice(datetime(2022, 5, 9, 5, 6))

In [ ]:
set_data.BO1.DATA.SINGLE_ARCHETYPE_SUMMARY_FRAME.loc[['W']]

In [ ]:
from data_fetching.utils.index_slice_helper import get_name_slice, get_color_slice

In [ ]:
set_data.BO1.DATA.CARD_HISTORY_FRAME.loc[slice('2022-04-29', '2022-05-01'), get_color_slice(''), get_name_slice({'Girder Goons', 'Angelic Observer', 'A Little Chat', 'Case the Joint'})]

In [ ]:
data = set_data.BO1.DATA.CARD_HISTORY_FRAME.loc(axis=0)[pd.IndexSlice[slice(None), get_color_slice(''), get_name_slice(None)]]
cheap = data[data['CMC'] <= 3]
expensive = data[data['CMC'] >= 5]

In [ ]:
cheap['Games Won'] = cheap['# GIH'] * cheap['GIH WR'] * 0.01
cheap_sums = cheap[['# GP', '# GIH', 'Games Won']].sum()
cheap_wr = cheap_sums['Games Won'] / cheap_sums['# GIH']
cheap_wr

In [ ]:
expensive['Games Won'] = expensive['# GIH'] * expensive['GIH WR'] * 0.01
expensive_sums = expensive[['# GP', '# GIH', 'Games Won']].sum()
expensive_wr = expensive_sums['Games Won'] / expensive_sums['# GIH']
expensive_wr

In [ ]:
for i in range(0, 9):
    by_cmc = data[data['CMC'] == i]
    by_cmc['Games Won'] = by_cmc['# GIH'] * by_cmc['GIH WR'] * 0.01
    sums = by_cmc[['# GP', '# GIH', 'Games Won']].sum()
    avg = by_cmc[['GIH WR']].mean()['GIH WR']
    wr = sums['Games Won'] / sums['# GIH'] * 100
    print(f"CMC: {i}, WR: {wr}  AVG: {avg}")

In [ ]:
examiner.get_archetype_playrate_history(['2C', '3C', 'WU', 'WG', 'UB', 'BR', 'RG', 'WUB', 'WUG', 'UBR', 'BRG', 'WRG'])

In [ ]:
examiner.get_archetype_playrate_history(['2C', '3C', 'WU', 'WG', 'UB', 'BR', 'RG', 'WUB', 'WUG', 'UBR', 'BRG', 'WRG']).diff()

In [ ]:
examiner.get_archetype_playrate_history(['2C', '3C', 'WU', 'WG', 'UB', 'BR', 'RG', 'WUB', 'WUG', 'UBR', 'BRG', 'WRG']).diff().diff()

# TODO

- Calculate archetype openess
 - GIH WR & ALSA based
 - 2.25 of a common per draft 
- Improve graphing capabilities
 - Modify Graphs so they also save an image to a cache folder.
 - Implement better graph titles and axes
 - Update functions to be more general, and have DataFrames piped into them.
- Move sets of files into subfolders based on functionality and relationship (DataFetch, Cards, Graphing, Utils, etc.)
- Better group settings and consts into one file/location.

# Data Graphing and Display

## Card Summary

In [ ]:
examiner.plot_card_summary('Jewel Thief')

In [ ]:
examiner.plot_card_summary('Disciplined Duelist')

In [ ]:
examiner.get_top('GIH WR', count=25, asc=False, deck_color='', card_color=None, card_rarity='C', play_lim=0.005)

In [ ]:
to_plot = [
    ('Murder', ''),
    ('Strangle', ''),
    ('Hold for Ransom', ''),
    ('Run Out of Town', ''),
    ('Make Disappear', ''),
    ('Witness Protection', ''),
    ('Sleep with the Fishes', ''),
    ('Strangle', 'BR'),
    ('Body Dropper', ''),
    ('Celestial Regulator', ''),
    ('Civil Servant', ''),
    ("Jetmir's Fixer", ''),
    ('Snooping Newsie', ''),
    ('Body Dropper', 'BR'),
    ('Celestial Regulator', 'UW'),
    ('Civil Servant', 'GW'),
    ("Jetmir's Fixer", 'GR'),
    ('Snooping Newsie', 'UB'),
    ("Spara's Adjudicators", ''),
    ('Glamorous Outlaw', ''),
    ('Rakish Revelers', ''),
    ('Masked Bandits', ''),
    ('Shattered Seraph', ''),
    ('Inspiring Overseer', ''),
    ("Raffine's Informant", ''),
    ('Gathering Throng', ''),
    ('Jewel Thief', ''),
    ('Echo Inspector', ''),
    ('Girder Goons', ''),
    ('Dapper Shieldmate', ''),
    ('Corrupt Court Official', ''),
    ('Caldaia Strongarm', ''),
    ('Backup Agent', ''),
    ('Mayhem Patrol', '')
]

for tup in to_plot:
    print(f"""Plotting "{tup[0]}"...""", end='')
    examiner.plot_card_summary(tup[0], colors=tup[1])
    print("     Done!")

## Card Pick Order Stats

In [ ]:
examiner.plot_pick_stats('Inspiring Overseer')

In [ ]:
examiner.plot_pick_stats('Inspiring Overseer', 3)

## Archetype Winrate History

In [ ]:
prettify_frame(examiner.get_archetype_winrate_history(['ALL', '2C', '3C', 'WU', 'WG', 'UB', 'BR', 'RG', 'WUB', 'WUG', 'UBR', 'BRG', 'WRG'], save=True))

In [ ]:
examiner.plot_archetype_winrate_history(['ALL', '2C', '3C', 'WU', 'WG', 'UB', 'BR', 'RG', 'WUB', 'WUG', 'UBR', 'BRG', 'WRG'])
examiner.plot_archetype_winrate_history(['ALL', '2C', 'WU', 'WG', 'UB', 'BR', 'RG'], pref='2C')
examiner.plot_archetype_winrate_history(['ALL', '3C', 'WUB', 'WUG', 'UBR', 'BRG', 'WRG'], pref='3C')

## Archetype Playrate History

In [ ]:
prettify_frame(examiner.get_archetype_playrate_history(['2C', '3C', 'WU', 'WG', 'UB', 'BR', 'RG', 'WUB', 'WUG', 'UBR', 'BRG', 'WRG'], save=True))

In [ ]:
examiner.plot_archetype_playrate_history(['2C', '3C', 'WU', 'WG', 'UB', 'BR', 'RG', 'WUB', 'WUG', 'UBR', 'BRG', 'WRG'])
examiner.plot_archetype_playrate_history(['WU', 'WG', 'UB', 'BR', 'RG'], 2, pref='2C')
examiner.plot_archetype_playrate_history(['WUB', 'WUG', 'UBR', 'BRG', 'WRG'], 3, pref='3C')

## Achetype Overviews

In [ ]:
examiner.plot_archetype_winrate_history(['ALL', '2C', 'WU', 'WG', 'UB', 'BR', 'RG'], pref='2C')
examiner.plot_archetype_winrate_history(['ALL', '3C', 'WUB', 'WUG', 'UBR', 'BRG', 'WRG'], pref='3C')
examiner.plot_archetype_playrate_history(['WU', 'WG', 'UB', 'BR', 'RG'], 2, pref='2C')
examiner.plot_archetype_playrate_history(['WUB', 'WUG', 'UBR', 'BRG', 'WRG'], 3, pref='3C')

## Card Pick Order Changes

In [ ]:
#TODO: Merge all of this functionality into one structure.
helper = PlotterHelper(examiner.DATA)
diff = examiner.compare_card_evaluations('2022-04-28', '2022-05-08')
commons = diff[diff['Rarity'] == 'C']
uncommons = diff[diff['Rarity'] == 'U']
SRT_TRG = 'ATA Change'

In [ ]:
frame = commons.sort_values(SRT_TRG, ascending=False).head(20)
helper.frame_to_png(frame, "HigherTakenCommons.png")

In [ ]:
frame = commons.sort_values(SRT_TRG, ascending=True).head(20)
helper.frame_to_png(frame, "LowerTakenCommons.png")

In [ ]:
frame = uncommons.sort_values(SRT_TRG, ascending=False).head(10)
helper.frame_to_png(frame, "HigherTakenUncommons.png")

In [ ]:
frame = uncommons.sort_values(SRT_TRG, ascending=True).head(10)
helper.frame_to_png(frame, "LowerTakenUncommons.png")